<a href="https://colab.research.google.com/github/lunathanael/chessnn/blob/main/zero_policy_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install chess
import chess
import math
import numpy as np
import tensorflow as tf
from typing import List
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, Dense, Flatten
from tensorflow.keras.models import Model
from threading import Thread, Event

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.3 MB/s eta 0:00:00


In [3]:
def residual_block(x, filters):
    """Create a residual block."""
    y = Conv2D(filters, kernel_size=3, padding='same')(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    y = Conv2D(filters, kernel_size=3, padding='same')(y)
    y = BatchNormalization()(y)
    y = Add()([y, x])
    y = ReLU()(y)
    return y

def make_network():
    # Input layer
    input_layer = Input(shape=(8, 8, 20))  # 8x8 grid with 20 features per cell

    # Body
    x = Conv2D(256, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = BatchNormalization()(x)
    # 19 residual blocks
    for _ in range(19):
        x = residual_block(x, 256)  # Each block has two convolutional layers with 256 filters, kernel size 3x3

    # Policy Head
    policy_head = Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    policy_head = BatchNormalization()(policy_head)
    policy_head = Conv2D(73, kernel_size=1)(policy_head)  # Output 73 policies

    # Value Head
    value_head = Conv2D(1, kernel_size=1, activation='relu')(x)
    value_head = BatchNormalization()(value_head)
    value_head = Flatten()(value_head)
    value_head = Dense(256, activation='relu')(value_head)
    value_head = Dense(1, activation='tanh')(value_head)  # Output single value

    # Create the model
    model = tf.keras.Model(inputs=input_layer, outputs=[value_head, policy_head])
    return model


def ConstantPolicyHead(shape):
    # Custom layer for policy head
    class CPHead(tf.keras.layers.Layer):
        def call(self, inputs):
            return tf.constant(1/73, shape=shape)

    return CPHead()

def ConstantValueHead():
    # Custom layer for value head
    class CVHead(tf.keras.layers.Layer):
        def call(self, inputs):
            return tf.constant(0.5, shape=(1,))

    return CVHead()

class make_uniform_network():
  def __init__(self):
    self._val =  np.full((1,1,), 0.5)
    self._policy = array2 = np.full((1, 8, 8, 73), 1/73)
  def predict(self, input):
    return self._val, self._policy


Helpers

In [4]:
class Config(object):

  def __init__(self):
    ### Self-Play
    self.num_actors = 1 # not enough ram <- 5000

    self.num_sampling_moves = 30
    self.max_moves = 512  # for chess and shogi, 722 for Go.
    self.num_simulations = 800

    # Root prior exploration noise.
    self.root_dirichlet_alpha = 0.3  # for chess, 0.03 for Go and 0.15 for shogi.
    self.root_exploration_fraction = 0.25

    # UCB formula
    self.pb_c_base = 19652
    self.pb_c_init = 1.25

    ### Training
    self.training_steps = int(100e3) # 700,000 take too long, temp
    self.checkpoint_interval = int(1e3)
    self.window_size = int(1e6)
    self.batch_size = 4096

    self.weight_decay = 1e-4
    self.momentum = 0.9
    # Schedule for chess and shogi, Go starts at 2e-2 immediately.
    self.learning_rate_schedule = {
        0: 2e-1,
        100e3: 2e-2,
        300e3: 2e-3,
        500e3: 2e-4
    }

In [5]:
class Node(object):

  def __init__(self, prior: float):
    self.visit_count = 0
    self.to_play = -1
    self.prior = prior
    self.value_sum = 0
    self.children = {}

  def expanded(self):
    return len(self.children) > 0

  def value(self):
    if self.visit_count == 0:
      return 0
    return self.value_sum / self.visit_count


In [6]:
def fen_to_repr(fen, repeats):
    # Split the FEN string to get the relevant parts
    parts = fen.split(' ')
    board_fen, player, castling, _, halfmove, fullmove = parts[:6]

    # 12 pieces, 4 cf, col, rep, half, total
    board = np.zeros((8, 8, 12 + 4 + 1 + 1 + 1 + 1), dtype=float)

    # Define piece order and mapping to layers
    piece_map = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5,
                 'P': 6, 'N': 7, 'B': 8, 'R': 9, 'Q': 10, 'K': 11}

    # Fill the board with pieces
    row = 0
    col = 0
    for char in board_fen:
        if char.isdigit():
            col += int(char)
        elif char == '/':
            row += 1
            col = 0
        else:
            board[row, col, piece_map[char]] = 1
            col += 1

    # Castling rights encoded in four binary planes
    board[:, :, 12] = 1 if 'K' in castling else 0  # White kingside
    board[:, :, 13] = 1 if 'Q' in castling else 0  # White queenside
    board[:, :, 14] = 1 if 'k' in castling else 0  # Black kingside
    board[:, :, 15] = 1 if 'q' in castling else 0  # Black queenside

    # Player color (1 for black, 0 for white)
    board[:, :, 16] = 1 if player == 'b' else 0

    # Position repetitions
    board[:, :, 17] = repeats

    board[:, :, 18] = float(halfmove)

    # Move number (as a real value)
    board[:, :, 19] = float(fullmove)

    return board

In [7]:
def square_to_coord(square):
  #       r           c
  return (square // 8, square % 8)

knight_moves = [
    (1, 2), (1, -2),
    (2, 1), (2, -1),
    (-1, 2), (-1, -2),
    (-2, 1), (-2, -1)
]

promotion_pieces = "nbr"
rows = "abcdefgh"

queen_moves = [
    (1, 1), (1, -1), (1, 0),
    (-1, 1), (-1, -1), (-1, 0),
    (0, 1), (0, -1)
]

def coord_to_uci(fr, fc, tr, tc):
  return rows[fc] + str(fr + 1) + rows[tc] + str(tr + 1)

def decode_action(action):

  #piece = self.board.piece_at(move.from_square)
  #fr, fc = square_to_coord(move.from_square)
  #tr, tc = square_to_coord(move.to_square)
  fr = action[0]
  fc = action[1]

  if action[2] < 9:
    piece_idx = action[2] // 3
    move_idx = action[2] % 3
    if fr == 1: # black promo
      tr = 0
    else:
      tr = 7
    if move_idx == 0:
      tc = fc
    elif move_idx == 1:
      tc = fc - 1
    else:
      tc = fc + 1
    return coord_to_uci(fr, fc, tr, tc) + promotion_pieces[piece_idx]

  if action[2] < 17:
    tr = fr + knight_moves[action[2] - 9][0]
    tc = fc + knight_moves[action[2] - 9][1]
  else:
    dist = (action[2] - 17) // 8 + 1
    status = (action[2] - 17) % 8
    tr = fr + (queen_moves[status][0] * dist)
    tc = fc + (queen_moves[status][1] * dist)

  return coord_to_uci(fr, fc, tr, tc)


class Environment(object):

  def __init__(self, env=None):
    if env == None:
      self.board = chess.Board(chess.STARTING_FEN)
      self.board_history = []
      self.repetitions = {}

      self.update_history()
    else:
      self.board = chess.Board.copy(env.board)
      self.board_history = env.board_history.copy()
      self.repetitions = env.repetitions.copy()

  def is_terminal(self):
    return self.board.is_game_over()

  def terminal_value(self, to_play):
    if self.board.is_checkmate():
      color = self.board.outcome.winner
      color = (color == chess.BLACK)
      return color == to_play
    else:
      return 0.5

  def generate_legal_moves(self):
    return self.board.generate_legal_moves()

  def encode_action(self, move): #12 piece types, 4 flags, 2 coords

    piece = self.board.piece_at(move.from_square)
    fr, fc = square_to_coord(move.from_square)
    tr, tc = square_to_coord(move.to_square)

    if move.promotion and move.promotion != 5: # 9 planes
      # move.promotion -> 2 : N, 3 : B, 4 : R, 5 : Q
      piece_idx = move.promotion - 2
      if fc == tc:
        move_idx = 0
      elif tc < fc: # left from white perspective
        move_idx = 1
      else:
        move_idx = 2
      plane = piece_idx * 3 + move_idx

    elif str(piece).lower() == 'n': # 8 planes
      for idx in range(len(knight_moves)):
        if fr + knight_moves[idx][0] == tr and fc + knight_moves[idx][1] == tc:
          plane = idx + 9
          break

    else: # 7 * 8 planes
      if fr < tr: # moving forward
        row_status = 0
      elif fr > tr: # back
        row_status = 1
      else: # horizontal movement
        row_status = 2

      if fc < tc: # moving right
        col_status = 0
      elif fc > tc: # left
        col_status = 1
      else: # vertical movement
        col_status = 2

      # row status and col status should never both be 2, means something stinky

      dist = max(abs(fr - tr), abs(fc - tc)) - 1
      plane = dist * 8 + (row_status * 3 + col_status) + 17

    # if decode_action((fr, fc, plane)) != str(move):
    #   print("found diff")
    #   print("action: ", fr, fc, plane)
    #   print("decoded: ", decode_action((fr, fc, plane)))
    #   print("Coords: ", fr, fc, tr, tc)
    #   print(move)
    #   print(str(move) == coord_to_uci(fr, fc, tr, tc))
    #   raise Exception("found differing in uci")

    return (fr, fc, plane)

  def apply(self, action):
    uci = decode_action(action)
    if action[0] == 1 and len(uci) == 4 and str(self.board.piece_at(action[0]*8+action[1])) == 'p':
      uci += 'q'
    if action[0] == 6 and len(uci) == 4 and str(self.board.piece_at(action[0]*8+action[1])) == 'P':
      uci += 'q'
    self.board.push_uci(uci)
    self.update_history()
    #print(self.board)

  def generate_legal_actions(self):
    moves = self.generate_legal_moves()
    actions = [self.encode_action(move) for move in moves]
    return actions

  def update_history(self):
    fen = self.board.fen()
    s_fen = fen.split(' ')
    cs_fen = ' '.join(s_fen[:-2])

    if cs_fen in self.repetitions:
      self.repetitions[cs_fen] += 1
      repeats = self.repetitions[cs_fen]
    else:
      self.repetitions[cs_fen] = 1
      repeats = 1

    halfmove_clock = self.board.halfmove_clock
    self.board_history.append((fen, repeats))

  def make_image(self, state_index=-1):
    fen, repeats= self.board_history[state_index]

    repr = fen_to_repr(fen, repeats)
    return repr

In [8]:
class Game(object):

  def __init__(self, history=None, environment=None):
    self.history = history or []
    self.child_visits = []
    self.num_actions = 4672  # action space size for chess
    self.environment = Environment(environment)

  def terminal(self):
    # Game specific termination rules.
    return self.environment.is_terminal()

  def terminal_value(self, to_play):
    # Game specific value.
    return self.environment.terminal_value()

  def legal_actions(self):
    # Game specific calculation of legal actions.
    return self.environment.generate_legal_actions()

  def clone(self):
    return Game(list(self.history), self.environment)

  def apply(self, action):
    self.history.append(action)
    self.environment.apply(action)

  def store_search_statistics(self, root):
    sum_visits = sum(child.visit_count for child in root.children.values())
    self.child_visits.append([
        root.children[a].visit_count / sum_visits if a in root.children else 0
        for a in range(self.num_actions)
    ])

  def make_image(self, state_index: int):
    # Game specific feature planes.
    return self.environment.make_image(state_index)

  def make_target(self, state_index: int):
    return (self.terminal_value(state_index % 2),
            self.child_visits[state_index])

  def to_play(self):
    return len(self.history) % 2


In [9]:
buffer_full_event = Event()

In [10]:
class ReplayBuffer(object):

  def __init__(self, config: Config):
    self.window_size = config.window_size
    self.batch_size = config.batch_size
    self.buffer = []

  def save_game(self, game):
    if len(self.buffer) > self.window_size:
      buffer_full_event.set()
      self.buffer.pop(0)
    self.buffer.append(game)
    print("Game saved")

  def sample_batch(self):
    # Sample uniformly across positions.
    print("Sample from: ", len(self.buffer))
    move_sum = float(sum(len(g.history) for g in self.buffer))
    games = np.random.choice(
        self.buffer,
        size=self.batch_size,
        p=[len(g.history) / move_sum for g in self.buffer])
    game_pos = [(g, np.random.randint(len(g.history))) for g in games]
    return [(g.make_image(i), g.make_target(i)) for (g, i) in game_pos]

In [64]:
class Network(object):

  def __init__(self, uniform_model: bool=False):
    if uniform_model:
      self.model = make_uniform_network()
    else:
      print("New network created.")
      self.model = make_network()

  def inference(self, image): # inference for SINGLE IMAGE
      # Run the neural network model to get predictions
      image = np.expand_dims(image, axis=0)
      value, policy_logits = self.model.predict(image, verbose=0)

      value = value[0] # The value output is a scalar representing the predicted game outcome
      policy_logits = policy_logits[0]

      return value, np.array(policy_logits)

  def batch_inference(self, images):
    # Run the neural network model to get predictions
    values, policy_logits = self.model.predict(images, verbose=0)

    return values, np.array(policy_logits)


  def get_weights(self):
    # Returns the weights of this network.
    return self.model.get_weights()

In [12]:
class SharedStorage(object):

  def __init__(self):
    self._networks = {}

  def latest_network(self) -> Network:
    if self._networks:
      return self._networks[max(self._networks.keys())]
    else:
      return Network(True)  # policy -> uniform, value -> 0.5

  def save_network(self, step: int, network: Network):
    self._networks[step] = network


In [13]:
# training is split into two independent parts: Network training and
# self-play data generation.
# These two parts only communicate by transferring the latest network checkpoint
# from the training to the self-play, and the finished games from the self-play
# to the training.
def zero(config: Config):
  storage = SharedStorage()
  replay_buffer = ReplayBuffer(config)
  run_selfplay(config, storage, replay_buffer)
  threads = [Thread(target=run_selfplay, args=(config, storage, replay_buffer))
            for _ in range(config.num_actors)]

  for t in threads:
    t.start()

  print("Self-play data generation launched.")

  train_network(config, storage, replay_buffer)

  return storage.latest_network()



In [14]:
##################################
####### Part 1: Self-Play ########


# Each self-play job is independent of all others; it takes the latest network
# snapshot, produces a game and makes it available to the training job by
# writing it to a shared replay buffer.
def run_selfplay(config: Config, storage: SharedStorage,
                 replay_buffer: ReplayBuffer):
  while True:
    network = storage.latest_network()
    game = play_game(config, network)
    replay_buffer.save_game(game)


# Each game is produced by starting at the initial board position, then
# repeatedly executing a Monte Carlo Tree Search to generate moves until the end
# of the game is reached.
def play_game(config: Config, network: Network):
  game = Game()
  while not game.terminal() and len(game.history) < config.max_moves:
    action, root = run_mcts(config, game, network)
    game.apply(action)
    game.store_search_statistics(root)
  return game


# Core Monte Carlo Tree Search algorithm.
# To decide on an action, we run N simulations, always starting at the root of
# the search tree and traversing the tree according to the UCB formula until we
# reach a leaf node.
def run_mcts(config: Config, game: Game, network: Network):
  root = Node(0)
  evaluate(root, game, network)
  add_exploration_noise(config, root)

  for _ in range(config.num_simulations):
    node = root
    scratch_game = game.clone()
    search_path = [node]

    while node.expanded():
      action, node = select_child(config, node)
      scratch_game.apply(action)
      search_path.append(node)

    value = evaluate(node, scratch_game, network)
    backpropagate(search_path, value, scratch_game.to_play())
  return select_action(config, game, root), root



def softmax_sample(visit_counts, temperature=10.0):
    counts, actions = zip(*visit_counts)
    # Apply softmax with temperature
    counts = np.array(counts)
    counts = counts / temperature  # Apply temperature scaling
    softmax_probs = np.exp(counts) / sum(np.exp(counts))
    # Sample an action based on the softmax probabilities
    rnd_idx = np.random.choice(len(actions), p=softmax_probs)
    action = actions[rnd_idx]
    return softmax_probs, action


def select_action(config: Config, game: Game, root: Node):
  visit_counts = [(child.visit_count, action)
                  for action, child in root.children.items()]
  if len(game.history) < config.num_sampling_moves:
    _, action = softmax_sample(visit_counts)
  else:
    _, action = max(visit_counts)
  return action


# Select the child with the highest UCB score.
def select_child(config: Config, node: Node):
  _, action, child = max((ucb_score(config, node, child), action, child)
                         for action, child in node.children.items())
  return action, child


# The score for a node is based on its value, plus an exploration bonus based on
# the prior.
def ucb_score(config: Config, parent: Node, child: Node):
  pb_c = math.log((parent.visit_count + config.pb_c_base + 1) /
                  config.pb_c_base) + config.pb_c_init
  pb_c *= math.sqrt(parent.visit_count) / (child.visit_count + 1)

  prior_score = pb_c * child.prior
  value_score = child.value()
  return prior_score + value_score


# We use the neural network to obtain a value and policy prediction.
def evaluate(node: Node, game: Game, network: Network):
  value, policy_logits = network.inference(game.make_image(-1))

  # Expand the node.
  node.to_play = game.to_play()

  policy = {a: math.exp(policy_logits[a]) for a in game.legal_actions()}
  policy_sum = sum(policy.values())
  for action, p in policy.items():
    node.children[action] = Node(p / policy_sum)
  return value


# At the end of a simulation, we propagate the evaluation all the way up the
# tree to the root.
def backpropagate(search_path: List[Node], value: float, to_play):
  for node in search_path:
    node.value_sum += value if node.to_play == to_play else (1 - value)
    node.visit_count += 1


# At the start of each search, we add dirichlet noise to the prior of the root
# to encourage the search to explore new actions.
def add_exploration_noise(config: Config, node: Node):
  actions = node.children.keys()
  noise = np.random.gamma(config.root_dirichlet_alpha, 1, len(actions))
  frac = config.root_exploration_fraction
  for a, n in zip(actions, noise):
    node.children[a].prior = node.children[a].prior * (1 - frac) + n * frac


######### End Self-Play ##########
##################################

##################################
####### Part 2: Training #########

class ZeroLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, lr_schedule):
        super(ZeroLearningRateSchedule, self).__init__()
        # Convert keys to integers
        self.lr_schedule = {int(k): v for k, v in lr_schedule.items()}
        self.lr_schedule_keys = sorted(self.lr_schedule)

    def __call__(self, step):
        learning_rate = self.lr_schedule[self.lr_schedule_keys[0]]
        for i in range(1, len(self.lr_schedule_keys)):
            if step < self.lr_schedule_keys[i]:
                break
            learning_rate = self.lr_schedule[self.lr_schedule_keys[i]]
        return learning_rate

    def get_config(self):
        return {'lr_schedule': self.lr_schedule}


def train_network(config: Config, storage: SharedStorage,
                  replay_buffer: ReplayBuffer):
  print("Waiting...")
  buffer_full_event.wait() # wait for full buffer

  network = Network()
  learning_rate_schedule = ZeroLearningRateSchedule(config.learning_rate_schedule)
  optimizer = tf.keras.optimizers.SGD(learning_rate_schedule, config.momentum)
  for i in range(config.training_steps):
    if i and i % config.checkpoint_interval == 0: # skip first checkpoint
      print("Checkpoint: ", i // config.checkpoint_interval)
      storage.save_network(i, network)
    batch = replay_buffer.sample_batch()
    update_weights(optimizer, network, batch, config.weight_decay)
  storage.save_network(config.training_steps, network)

def update_weights(optimizer: tf.keras.optimizers, network: Network, batch,
                   weight_decay: float):
  loss = 0
  for image, (target_value, target_policy) in batch:
    value, policy_logits = network.inference(image)
    loss += (
        tf.losses.mean_squared_error(value, target_value) +
        tf.nn.softmax_cross_entropy_with_logits(
            logits=policy_logits, labels=target_policy))

  for weights in network.get_weights():
    loss += weight_decay * tf.nn.l2_loss(weights)

  optimizer.minimize(loss)


######### End Training ###########
##################################


def launch_job(f, *args):
  f(*args)

In [65]:
import cProfile
config = Config()
game = Game()
nn = Network()

with cProfile.Profile() as pr:
  pass
  #run_mcts(config, game, nn)

New network created.


In [66]:
image = Game().make_image(-1)
images = np.repeat([image], 4, axis = 0)

In [ ]:
import time

batch_size = 1
for _ in range(10):
  images = np.repeat([image], batch_size, axis=0)
  start = time.time()
  for _ in range(5):
    nn.batch_inference(images)
  end = time.time()
  tot = end - start
  print("Batch size: ", batch_size, ", total time: ", tot, "per image: ", tot / (5*batch_size))
  batch_size *= 2


Batch size:  1 , total time:  0.8363618850708008 per image:  0.16727237701416015
Batch size:  2 , total time:  0.9925093650817871 per image:  0.0992509365081787
Batch size:  4 , total time:  1.5382449626922607 per image:  0.07691224813461303
Batch size:  8 , total time:  2.502981662750244 per image:  0.06257454156875611
Batch size:  16 , total time:  5.374805450439453 per image:  0.06718506813049316
Batch size:  32 , total time:  8.913215160369873 per image:  0.055707594752311705
Batch size:  64 , total time:  23.94163727760315 per image:  0.07481761649250984
Batch size:  128 , total time:  41.41246843338013 per image:  0.06470698192715645
Batch size:  256 , total time:  86.44875288009644 per image:  0.06753808818757534


In [50]:
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)
nn.inference(image)

1/1 [==============================] - 0s 109ms/step


(array([0.73793906], dtype=float32),
 array([[[-1.58127356e+00,  4.11895573e-01, -4.11278784e-01, ...,
          -3.56388897e-01,  6.53102756e-01,  9.71852422e-01],
         [-1.65438128e+00,  3.92223477e-01, -8.23434949e-01, ...,
          -6.47575259e-01,  3.86123240e-01,  6.37032866e-01],
         [-2.03502178e+00,  1.17488456e+00, -9.11915064e-01, ...,
          -3.86198848e-01,  6.65799141e-01,  3.04536015e-01],
         ...,
         [-9.01853561e-01,  9.52573299e-01, -1.90259010e-01, ...,
          -6.36822104e-01,  1.17242253e+00,  1.23982236e-01],
         [-1.30446613e+00,  1.08352983e+00, -5.80132231e-02, ...,
          -2.71797180e-01,  1.04940069e+00,  9.14379179e-01],
         [-6.66254580e-01,  9.44324851e-01,  3.25394988e-01, ...,
          -6.79210052e-02,  5.79576135e-01,  4.42014426e-01]],
 
        [[-1.35103846e+00,  3.65273297e-01, -3.22516650e-01, ...,
          -3.88961285e-01,  5.45826733e-01,  2.23604727e+00],
         [-1.82733119e+00,  5.73998034e-01, -7.975

In [ ]:
import cProfile
config = Config()
game = Game()
nn = Network()

with cProfile.Profile() as pr:
  #run_mcts(config, game, nn)
  image = Game().make_image(-1)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)
  nn.inference(image)

In [16]:
pr.dump_stats('restats')

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
#p.sort_stats('tottime')
#p.print_callees()
#p.print_stats()
p.sort_stats(1)
p.print_stats()

In [20]:
import cProfile
config = Config()
network = Network(True)

with cProfile.Profile() as pr:
  run_mcts()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
P . . . . . . .
. P P P P P P P
R N B Q K B N R
r . b q k b n r
p p p p p p p p
. . n . . . . .
. . . . . . . .
. . . . . . . .
P . . . . . . .
. P P P P P P P
R N B Q K B N R
r . b q k b n r
p p p p p p p p
. . n . . . . .
. . . . . . . .
. . . . . . . .
P . N . . . . .
. P P P P P P P
R . B Q K B N R
r . b q k b n r
p p p p p p p p
. . . . . . . .
. . . . n . . .
. . . . . . . .
P . N . . . . .
. P P P P P P P
R . B Q K B N R
r . b q k b n r
p p p p p p p p
. . . . . . . .
. . . . n . . .
. . . . . . . .
P P N . . . . .
. . P P P P P P
R . B Q K B N R


KeyboardInterrupt: 

In [ ]:
# config = Config()
# network_1 = zero(config)